In [1]:
import duckdb
import os
from dotenv import load_dotenv, find_dotenv
from deltalake import DeltaTable, write_deltalake

load_dotenv(find_dotenv())

con = duckdb.connect()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_MINIO")
AWS_SECRET_KEY_ID = os.getenv("AWS_SECRET_KEY_MINIO")
HOST_MINIO = os.getenv("HOST_MINIO")
PORT_MINIO = os.getenv("PORT_MINIO")

con.sql(f""" CREATE SECRET secret1 (
            TYPE S3,
            KEY_ID '{AWS_ACCESS_KEY_ID}',
            SECRET '{AWS_SECRET_KEY_ID}',
            REGION 'us-east-1',
            ENDPOINT '{HOST_MINIO}:{PORT_MINIO}',
            URL_STYLE 'path',
            USE_SSL 'false'

        );
    """)

storage_options = {
    "AWS_ACCESS_KEY_ID": f"{AWS_ACCESS_KEY_ID}",
    "AWS_SECRET_ACCESS_KEY": f"{AWS_SECRET_KEY_ID}",
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:{PORT_MINIO}",
    "AWS_allow_http": "true",
    "AWS_REGION": "us-east-1",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
}

In [2]:
path_silver = "s3://silver/uff/projeto_comex"
path_gold = "s3://gold/uff/projeto_comex"

In [31]:
# Executa a consulta SQL para obter os dados e a descrição da tabela
dim_ncm = con.sql(f"""
                  
                  WITH silver_ncm_ex AS
                  (
                      select distinct
                        FK_NCM,
                        NOME_NCM,
                      FROM delta_scan('{path_silver}/EXP')
                  ),
                  silver_ncm_im AS
                  (
                      select distinct
                        FK_NCM,
                        NOME_NCM,
                    FROM delta_scan('{path_silver}/IMP')
                  )
                  SELECT * FROM( 
                    select
                      *
                    from silver_ncm_ex
                    union
                    select
                      *
                    from silver_ncm_im
                  ) A
                  WHERE A.FK_NCM NOT IN(
                    SELECT DISTINCT FK_NCM FROM delta_scan('{path_gold}/dim_ncm')
                  )                   
                  """).to_arrow_table()



In [18]:
table_path = f"{path_gold}/dim_ncm"

In [ ]:
if len(dim_ncm) > 0:
    write_deltalake(
        f"{path_gold}/dim_ncm",
        dim_ncm,
        mode="append",
        storage_options=storage_options,
    )

In [32]:
con.close()